In [1]:
#install dependencies
!pip install torch torchvision scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
#import libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import KFold
import numpy as np
import os
from PIL import Image, UnidentifiedImageError


import kagglehub
import torchvision.models as models

In [3]:
#preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [4]:
#load and label data

# Download dataset from KaggleHub

dataset_path = kagglehub.dataset_download("zaidpy/oral-cancer-dataset")
print("Dataset path:", dataset_path)



100%|██████████| 315M/315M [00:08<00:00, 37.4MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/zaidpy/oral-cancer-dataset/versions/2


In [5]:
import os

for root, dirs, files in os.walk(dataset_path):
    print(f"\n {root}")
    for d in dirs:
        print("   folder:", d)
    for f in files:
        print("   file:", f)



 /root/.cache/kagglehub/datasets/zaidpy/oral-cancer-dataset/versions/2
   folder: Oral cancer Dataset 2.0
   folder: Oral Cancer

 /root/.cache/kagglehub/datasets/zaidpy/oral-cancer-dataset/versions/2/Oral cancer Dataset 2.0
   folder: OC Dataset kaggle new

 /root/.cache/kagglehub/datasets/zaidpy/oral-cancer-dataset/versions/2/Oral cancer Dataset 2.0/OC Dataset kaggle new
   folder: CANCER
   folder: NON CANCER

 /root/.cache/kagglehub/datasets/zaidpy/oral-cancer-dataset/versions/2/Oral cancer Dataset 2.0/OC Dataset kaggle new/CANCER
   file: 247.jpeg
   file: 331.jpeg
   file: 102.jpeg
   file: 429.jpeg
   file: 029.jpeg
   file: 432.jpeg
   file: 408.jpeg
   file: 353.jpeg
   file: 073.jpeg
   file: 010.jpeg
   file: 494.jpeg
   file: 415.jpeg
   file: 466.jpeg
   file: 077.jpeg
   file: 370.jpeg
   file: 240.jpeg
   file: 040.jpeg
   file: 438.jpeg
   file: 214.jpeg
   file: 033.jpeg
   file: 445.jpeg
   file: 056.jpeg
   file: 450.jpeg
   file: 315.jpeg
   file: 486.jpeg
   file:

In [6]:
label_map = {"CANCER": 1, "NON CANCER": 0}

def load_image_paths():
    base_paths = [
        "/kaggle/input/oral-cancer-dataset/Oral cancer Dataset 2.0/OC Dataset kaggle new",
        "/kaggle/input/oral-cancer-dataset/Oral Cancer/Oral Cancer Dataset"
    ]

    image_paths, labels = [], []
    for base in base_paths:
        for label_name, label_value in label_map.items():
            full_path = os.path.join(base, label_name)
            if not os.path.exists(full_path):
                print(f"Skipping (not found): {full_path}")
                continue
            print(f"Loading from: {full_path}")
            for fname in os.listdir(full_path):
                if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                    image_paths.append(os.path.join(full_path, fname))
                    labels.append(label_value)
    return image_paths, labels


In [10]:
# image_paths, labels = load_image_paths(dataset_path)

# print("Sample image paths:", image_paths[:5])
# print("Total images loaded:", len(image_paths))
# print("Total labels loaded:", len(labels))

In [7]:
class OralCancerDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, label
        except (UnidentifiedImageError, OSError) as e:
            print(f"Skipping corrupted image: {img_path}")
            # Option 1: return a dummy image (only if batching requires it)
            # Option 2 (preferred): raise IndexError to signal skipping
            raise IndexError  # Will be caught by DataLoader with `drop_last=True`



In [8]:
#defining transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:
# def filter_valid_images(image_paths, labels):
#     valid_paths = []
#     valid_labels = []
#     for path, label in zip(image_paths, labels):
#         try:
#             with Image.open(path) as img:
#                 img.verify()  # Checks if it's readable without loading it
#             valid_paths.append(path)
#             valid_labels.append(label)
#         except Exception as e:
#             print(f"Corrupt image skipped: {path}")
#     return valid_paths, valid_labels

# image_paths, labels = load_image_paths()
# image_paths, labels = filter_valid_images(image_paths, labels)

# print(f"Valid images after filtering: {len(image_paths)}")


In [15]:
#cross validation and training

kf = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold, (train_idx, val_idx) in enumerate(kf.split(image_paths)):
    print(f"\n--- Fold {fold+1} ---")

    train_data = OralCancerDataset([image_paths[i] for i in train_idx],
                                   [labels[i] for i in train_idx],
                                   transform=transform)
    val_data = OralCancerDataset([image_paths[i] for i in val_idx],
                                 [labels[i] for i in val_idx],
                                 transform=transform)

    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=16, shuffle=False)

    # Load pretrained ResNet50 and modify classifier
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)  # Binary output
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(10):
        model.train()
        total_loss = 0

        for imgs, labels_ in train_loader:
            imgs, labels_ = imgs.to(device), labels_.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels_)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for imgs, labels_ in val_loader:
                imgs, labels_ = imgs.to(device), labels_.to(device)
                outputs = model(imgs)
                preds = torch.argmax(outputs, dim=1)
                y_true.extend(labels_.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        acc = np.mean(np.array(y_true) == np.array(y_pred))
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}, Val Accuracy = {acc:.4f}")

    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("AUROC:", roc_auc_score(y_true, y_pred))



--- Fold 1 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1: Loss = 22.8963, Val Accuracy = 0.9438
Epoch 2: Loss = 11.1401, Val Accuracy = 0.9645
Epoch 3: Loss = 7.8777, Val Accuracy = 0.9497
Epoch 4: Loss = 5.9949, Val Accuracy = 0.9615
Epoch 5: Loss = 3.6480, Val Accuracy = 0.9704
Epoch 6: Loss = 5.6223, Val Accuracy = 0.9645
Epoch 7: Loss = 6.7576, Val Accuracy = 0.9586
Epoch 8: Loss = 1.4246, Val Accuracy = 0.9793
Epoch 9: Loss = 2.1881, Val Accuracy = 0.9675
Epoch 10: Loss = 1.1577, Val Accuracy = 0.9556
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.95       144
           1       0.93      1.00      0.96       194

    accuracy                           0.96       338
   macro avg       0.96      0.95      0.95       338
weighted avg       0.96      0.96      0.96       338

Confusion Matrix:
 [[129  15]
 [  0 194]]
AUROC: 0.9479166666666667

--- Fold 2 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1: Loss = 21.3730, Val Accuracy = 0.9349
Epoch 2: Loss = 9.2602, Val Accuracy = 0.8964
Epoch 3: Loss = 4.9639, Val Accuracy = 0.9556
Epoch 4: Loss = 7.2593, Val Accuracy = 0.9645
Epoch 5: Loss = 7.7225, Val Accuracy = 0.9497
Epoch 6: Loss = 4.3658, Val Accuracy = 0.9586
Epoch 7: Loss = 1.4548, Val Accuracy = 0.9645
Epoch 8: Loss = 3.2260, Val Accuracy = 0.9556
Epoch 9: Loss = 2.6242, Val Accuracy = 0.9556
Epoch 10: Loss = 2.9911, Val Accuracy = 0.9527
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       128
           1       0.96      0.96      0.96       210

    accuracy                           0.95       338
   macro avg       0.95      0.95      0.95       338
weighted avg       0.95      0.95      0.95       338

Confusion Matrix:
 [[120   8]
 [  8 202]]
AUROC: 0.949702380952381

--- Fold 3 ---


...

Epoch 1: Loss = 21.1776, Val Accuracy = 0.9349
Epoch 2: Loss = 9.3011, Val Accuracy = 0.9586
Epoch 3: Loss = 6.5527, Val Accuracy = 0.9349
Epoch 4: Loss = 6.9988, Val Accuracy = 0.9467
Epoch 5: Loss = 5.9015, Val Accuracy = 0.9527
Epoch 6: Loss = 2.0472, Val Accuracy = 0.9615
Epoch 7: Loss = 1.7421, Val Accuracy = 0.9467
Epoch 8: Loss = 2.2133, Val Accuracy = 0.9645
Epoch 9: Loss = 1.6823, Val Accuracy = 0.9408
Epoch 10: Loss = 6.6454, Val Accuracy = 0.8846
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.98      0.87       132
           1       0.99      0.82      0.90       206

    accuracy                           0.88       338
   macro avg       0.88      0.90      0.88       338
weighted avg       0.91      0.88      0.89       338

Confusion Matrix:
 [[130   2]
 [ 37 169]]
AUROC: 0.9026184171815239

--- Fold 4 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1: Loss = 23.4428, Val Accuracy = 0.9053
Epoch 2: Loss = 11.7869, Val Accuracy = 0.9556
Epoch 3: Loss = 5.2923, Val Accuracy = 0.9645
Epoch 4: Loss = 8.4770, Val Accuracy = 0.9527
Epoch 5: Loss = 3.9017, Val Accuracy = 0.9734
Epoch 6: Loss = 4.7942, Val Accuracy = 0.9793
Epoch 7: Loss = 4.7354, Val Accuracy = 0.9556
Epoch 8: Loss = 2.5330, Val Accuracy = 0.9763
Epoch 9: Loss = 1.9607, Val Accuracy = 0.9911
Epoch 10: Loss = 5.2123, Val Accuracy = 0.9822
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       152
           1       0.99      0.97      0.98       186

    accuracy                           0.98       338
   macro avg       0.98      0.98      0.98       338
weighted avg       0.98      0.98      0.98       338

Confusion Matrix:
 [[151   1]
 [  5 181]]
AUROC: 0.9832696661007357

--- Fold 5 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1: Loss = 23.4142, Val Accuracy = 0.9467
Epoch 2: Loss = 7.3210, Val Accuracy = 0.9497
Epoch 3: Loss = 7.7936, Val Accuracy = 0.9645
Epoch 4: Loss = 4.9012, Val Accuracy = 0.9438
Epoch 5: Loss = 5.0660, Val Accuracy = 0.9675
Epoch 6: Loss = 3.4672, Val Accuracy = 0.9704
Epoch 7: Loss = 4.7478, Val Accuracy = 0.9704
Epoch 8: Loss = 1.4585, Val Accuracy = 0.9615
Epoch 9: Loss = 1.2672, Val Accuracy = 0.9675
Epoch 10: Loss = 2.3945, Val Accuracy = 0.9556
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.91      0.95       144
           1       0.94      0.99      0.96       194

    accuracy                           0.96       338
   macro avg       0.96      0.95      0.95       338
weighted avg       0.96      0.96      0.96       338

Confusion Matrix:
 [[131  13]
 [  2 192]]
AUROC: 0.9497064719358533


In [16]:
#saving model

torch.save(model.state_dict(), f"resnet50_oral_cancer_fold{fold+1}.pth")
